## Load Packages

In [1]:
using Roots

include("printmat.jl")

printmatDate

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# Interest Rate vs (Zero Coupon) Bond Price

Recall: the bond price (for maturity $m$) and the the effective interest rate follow

$B(m) = [1+Y(m)]^{-m}$

Instead, with a continuously compounded interest rate we have 

$B(m) = e^{-m y(m)}$

In [3]:
function BillPrice(Y,m)             
    B = (1+Y)^(-m)
    return B
end
function BillPrice2(y,m)             
    B = exp(-m*y)
    return B
end

BillPrice2 (generic function with 1 method)

In [4]:
Ym  = 0:0.01:0.07             #consider different interest rates
B1  = BillPrice.(Ym,1)        #1-year zero coupon bond 
B10 = BillPrice.(Ym,10)       #10-year

println("let's plot it")

let's plot it


In [5]:
plot(Ym,B1,color=:red,linewidth=2,label="1y")
plot!(Ym,B10,color=:green,line=(:dash,2),label="10y")
title!("Bond prices and rates")
xlabel!("Interest rate")
ylabel!("Bond price")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.01 
 
 
 0.02 
 
 
 0.03 
 
 
 0.04 
 
 
 0.05 
 
 
 0.06 
 
 
 0.07 
 
 
 0.6 
 
 
 0.7 
 
 
 0.8 
 
 
 0.9 
 
 
 1.0 
 
 
 Bond prices and rates 
 
 
 Interest rate 
 
 
 Bond price 
 
 
 
 
 
 
 
 1y 
 
 
 
 10y

In [6]:
m  = 0.5                              #maturity
B  = 0.95                             #bond price
Y  = B^(-1/m) - 1                     #effective interest rate
y  = -log(B)/m                        #continuously compounded interest rate

printlnPs("Effective and continuously compounded interest rates ($m-year bond with price $B): ",[Y y])

Effective and continuously compounded interest rates (0.5-year bond with price 0.95):      0.108     0.103


# Bond Returns (Zero Coupon Bonds)

Let $B_0$ denote the price of a bond in period $0$ and $B_1$ the price of the same bond one period later. (For simplicity, we skip the notation for maturity.)

The *log* return of holding this bond is clearly $\ln (B_1/B_0)$

The cell below works with continuously compounded interest rates.

In [7]:
y₀ = 0.005                      #(continuously compounded) interest rate before
y₁ = 0.015                      #interest rate after  

B₀ = BillPrice2.(y₀,[1 10])     #bond prices (1-year and 10 year) before
B₁ = BillPrice2.(y₁,[1 10])     #bond prices (1-year and 10 year) after

R₁ = log.(B₁./B₀)                #return

printlnPs("Bonds:  1y    10y\n")
printlnPs("all interest rates change from $y₀ to $y₁,")
printlnPs("so the bond prices change from \n",B₀)
printlnPs("to\n",B₁)
printlnPs("which gives the returns \n",R₁)

Bonds:  1y    10y

all interest rates change from 0.005 to 0.015,
so the bond prices change from 
     0.995     0.951
to
     0.985     0.861
which gives the returns 
    -0.010    -0.100


# Forward Rates

From the forward-spot parity, the forward price of a bond (delivered in $m$ and maturing in $n$) is

$F=\left[  1+Y(m)\right]  ^{m}B(n)=B(n)/B(m)$.

A foward interest rate can then be defined as 

$\Gamma(m,n)  =F^{-1/(n-m)}-1$

In [8]:
function ForwardRate(Ym,m,Yn,n)           #forward rate, assuming n > m
    Bm = (1+Ym)^(-m) 
    Bn = (1+Yn)^(-n)
    F  = Bn/Bm
    Γ  = F^(-1/(n-m)) - 1
    return Γ
end

ForwardRate (generic function with 1 method)

In [9]:
m  = 0.5
n  = 0.75
Ym = 0.04
Yn = 0.05

Γ = ForwardRate(Ym,m,Yn,n)
printlnPs("\nImplied forward rate (from $m to $n years ahead): ",Γ)


Implied forward rate (from 0.5 to 0.75 years ahead):      0.070


# Coupon Bond Prices

Recall, the coupon bond price $P$ is the present value of the future cash flows $cf_k$

$P = \sum_{k=1}^{K} \frac{cf_{k}}{\left[  1+Y(m_{k})\right]  ^{m_{k}}}$

The BondPrice3(Y,cf,m) function (see below) can handle both the case when $Y$ is a vector with different values for different maturities and when Y is a scalar (same interest rate for all maturities).

In [10]:
"""
Y:  scalar or K vector of interest rates
cf: scalar or K vector of cash flows
m: K vector of times for the cash flows
"""
function BondPrice3(Y,cf,m)              #cf is a vector of all cash flows at times m
    cdisc = cf./((Y.+1).^m)                #c/(1+y1)^m1 + c/(1+y2)^m2 + ...
    P     = sum(cdisc)                     #price
    return P
end

BondPrice3

In [11]:
Y = [0.053;0.054]
c  = 0.06
P  =  BondPrice3(Y,[c;c+1],[1;2])
printlnPs("\nPrice of 2y bond with $c coupon when the spot rates are $Y: ",P)

Y  = [0.06;0.091]
c  = 0.09
P  =  BondPrice3(Y,[c;c+1],[1;2])
printlnPs("\nPrice of a 2y (almost) par bond. It has a $c coupon and spot rates are $Y. ",P)


Price of 2y bond with 0.06 coupon when the spot rates are [0.053, 0.054]:      1.011

Price of a 2y (almost) par bond. It has a 0.09 coupon and spot rates are [0.06, 0.091].      1.001


# Yield to Maturity

The yield to maturity is the $\theta$ that solves

$P = \sum_{k=1}^{K} \frac{cf_{k}}{(1+\theta)  ^{m_{k}}}$

We typically have to find $\theta$ by a numerical method.

In [12]:
c = 0.04
Y = 0.03                   #all spot rates are 3%    

P = BondPrice3(Y,[c;c+1],[1;2])
ytm = fzero(y->BondPrice3(y,[c;c+1],[1;2])-P,[-0.1;0.1])   #solving for ytm

printlnPs("Price of 2-year $c coupon bond when all spot rates are $Y: ",P)
printlnPs("\nytm of this bond: ",ytm)

Price of 2-year 0.04 coupon bond when all spot rates are 0.03:      1.019

ytm of this bond:      0.030


In [13]:
m  = [1;2;3]                  #3-year annuity
Y  = [0.04;0.05;0.06]         #spot interest rates 
c  = [1;1;1]

P  = BondPrice3(Y,c,m)
ytm = fzero(y->BondPrice3(y,c,m)-P,[-0.2;0.2])

printlnPs("\nytm of 3y annuity when the spot rates are $Y: ",ytm)


ytm of 3y annuity when the spot rates are [0.04, 0.05, 0.06]:      0.053


# Bootstrapping

Recall: with information about coupons $c(k)$ and coupon bond price $P(m)$, we solve for the implied zero coupon bond prices $B(s)$ from

$
\begin{bmatrix}
P(1)\\
P(2)
\end{bmatrix} =
\begin{bmatrix}
c(1)+1 & 0 \\
c(2) & c(2)+1
\end{bmatrix}
\begin{bmatrix}
B(1)\\
B(2)
\end{bmatrix}
$

In [14]:
c  = [0;0.06]                    #coupon rates of the bonds (here 2)
P  = [0.95;1.01]                 #coupon bond prices
m  = [1;2]                       #time of coupon payments

n = length(c)                           
cfMat = eye(n)
for i = 1:n, j = 1:i                      #fill the cfMat matrix 
    cfMat[i,j] = cfMat[i,j] + c[i]
end    

println("The cash flow matrix")
printmat(cfMat)

println("Solve P = cfMat*B for B (implied zero-coupon bond prices):")
B = cfMat\P
printmat(B)

Y = zeros(n)                   #solve for the implied spot rates
for i = 1:n
    Y[i] = B[i]^(-1/m[i]) - 1
end    
println("Implied spot interest rates:")
printmat(Y)

The cash flow matrix
     1.000     0.000
     0.060     1.060

Solve P = cfMat*B for B (implied zero-coupon bond prices):
     0.950
     0.899

Implied spot interest rates:
     0.053
     0.055



# Estimating Yield Curve with Regression Analysis

Recall: with a quadratic discount function

$B(m)=a_{0}+a_{1}m+a_{2}m^{2}$,

we can write the coupon bond price 

$P\left(  m_{K}\right)   =\sum_{k=1}^{K}B(m_{k})c+B(m_{K})$
as 

$P\left(  m_{K}\right)   =\sum_{k=1}^{K}\left(  a_{0}+a_{1}m_{k}+a_{2}m_{k}^{2}\right)  c+\left(
a_{0}+a_{1}m_{K}+a_{2}m_{K}^{2}\right)$

Collect terms as 

$
P\left(  m_{K}\right)  =a_{0}\underset{\text{term 0}}{\underbrace{(Kc+1)}
}+a_{1}\underset{\text{term 1}}{\underbrace{(c
{\textstyle\sum_{k=1}^{K}}
m_{k}+m_{K})}}+a_{2}\underset{\text{term 2}}{\underbrace{(c
{\textstyle\sum_{k=1}^{K}}
m_{k}^{2}+m_{K}^{2})}}$

We estimate $(a_0,a_1,a_2)$ by using the "terms" as regressors.


In [15]:
x = zeros(n,3)                      #create regressors for quadratic model: 3 columns
for i = 1:n                         #x[i,j] is for bond i, regressor j
    x[i,1] = m[i]*c[i] + 1
    x[i,2] = c[i]*sum(m[1]:m[i]) + m[i]
    x[i,3] = c[i]*sum((m[1]:m[i]).^2) + m[i]^2
end    

println("regressors:")
println("    term 0     term 1   term 2")
printmat(x)

a = x\P                           #regress Bc on x
println("regression coefficients in B")
printmat(a)

m = 1:5
B = a[1] .+ a[2]*m + a[3]*m.^2      #fitted discount function
println("maturities    fitted B")
printmat([m B])

print("Check that m=1 and m=2 give the same B as in the bootstrap")

regressors:
    term 0     term 1   term 2
     1.000     1.000     1.000
     1.120     2.180     4.300

regression coefficients in B
     0.693
     0.411
    -0.154

maturities    fitted B
     1.000     0.950
     2.000     0.899
     3.000     0.540
     4.000    -0.126
     5.000    -1.101

Check that m=1 and m=2 give the same B as in the bootstrap